## LLM + PROMPT = LLMChain

In [1]:
import os
import openai
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

from dotenv import load_dotenv

load_dotenv()

def get_secret(name):
    if name in os.environ:
        return os.environ[name]
    return os.getenv(name)

# create .env in project dir and set api keys there
openai.api_key = get_secret('OPENAI_API_KEY')

 
prompt = PromptTemplate(
    input_variables=["city"],
    template="Describe a perfect day in {city}?",
)
 


llmchain = LLMChain(llm=OpenAI(model_name="gpt-3.5-turbo-0301", temperature=0.0) , prompt=prompt)
llmchain.run("Paris")

/Users/uan/anaconda3/envs/agents/lib/python3.10/site-packages/langchain/llms/openai.py:171: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/uan/anaconda3/envs/agents/lib/python3.10/site-packages/langchain/llms/openai.py:740: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


"As an AI language model, I don't have personal preferences or experiences. However, here's an example of a perfect day in Paris:\n\nStart the day with a croissant and coffee at a local café, then head to the Eiffel Tower to take in the stunning views of the city. Next, visit the Louvre Museum to see some of the world's most famous art, including the Mona Lisa.\n\nFor lunch, enjoy a picnic in the Luxembourg Gardens, complete with fresh baguettes, cheese, and wine. Afterward, take a stroll through the charming streets of Montmartre, stopping to admire the Sacré-Cœur Basilica and browse the local shops.\n\nIn the evening, indulge in a delicious French dinner at a Michelin-starred restaurant, followed by a show at the Moulin Rouge. End the night with a romantic walk along the Seine River, taking in the beautiful lights of the city."

## Langchain chains, example LLMMathChain

In [2]:
import json
import paths
from langchain.llms import OpenAIChat

from langchain.chains import LLMMathChain
math_chain = LLMMathChain(llm=OpenAI(model_name="gpt-3.5-turbo-0301", temperature=0.0))


math_chain.run("""I have 5 apple trees. Each apple tree has 3 apples growing on it.
How many apples are there on 5 apple trees?""")

/Users/uan/anaconda3/envs/agents/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


'Answer: 15'

In [3]:
math_chain.run("""I have 5 apple trees. Each apple tree gives 3 apples each year.
How many apples I can collect after 7 years?""")

'Answer: 105'

In [3]:
math_chain = LLMMathChain(llm=OpenAI(model_name="gpt-3.5-turbo-0301", temperature=0.0), verbose=True)
math_chain.run("""I have 5 apple trees. Each apple tree gives 3 apples each year.
How many apples I can collect after 7 years? I also eat 8 apples every 3 years""")



> Entering new LLMMathChain chain...
I have 5 apple trees. Each apple tree gives 3 apples each year.
How many apples I can collect after 7 years? I also eat 8 apples every 3 years```text
(5 * 3 * 7) - (8 * (7 // 3))
```
...numexpr.evaluate("(5 * 3 * 7) - (8 * (7 // 3))")...

Answer: 89
> Finished chain.


'Answer: 89'

In [4]:
(5 * 3 * 7) - (8 * (7 // 3))

89

In [6]:
# math_chain = LLMMathChain(llm=OpenAI(model_name="gpt-3.5-turbo-0301", temperature=0.0, verbose=True))
# math_chain.run("""I have 5 apple trees. Each apple tree gives 3 apples each year.
# How many apples I can collect after 7 years if I lose 7 apples on 3rd year ?""")

## chain =  LLM  + Solidity compiler(check if is compilable)

In [7]:
! solc-select install 0.8.9
! solc-select use 0.8.9

Installing '0.8.9'...
Version '0.8.9' installed.
Switched global version to 0.8.9


In [5]:
# copied from https://python.langchain.com/en/latest/modules/chains/generic/custom_chain.html

from __future__ import annotations

from typing import Any, Dict, List, Optional

from pydantic import Extra

from langchain.base_language import BaseLanguageModel
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)
from langchain.chains.base import Chain
from langchain.prompts.base import BasePromptTemplate


class MyCustomChain(Chain):
    
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: BaseLanguageModel
    output_key: str = "text"  #: :meta private:

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)
        
        solidity_function = inputs['solidity_function']
        if not_compilable(solidity_function):
            return {self.output_key: "not compilable"}
        
        # Whenever you call a language model, or another chain, you should pass
        # a callback manager to it. This allows the inner run to be tracked by
        # any callbacks that are registered on the outer run.
        # You can always obtain a callback manager for this by calling
        # `run_manager.get_child()` as shown below.
        response = self.llm.generate_prompt(
            [prompt_value],
            callbacks=run_manager.get_child() if run_manager else None
        )

        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        if run_manager:
            run_manager.on_text("Log something about this run")
        
        return {self.output_key: response.generations[0][0].text}

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)
        
        # Whenever you call a language model, or another chain, you should pass
        # a callback manager to it. This allows the inner run to be tracked by
        # any callbacks that are registered on the outer run.
        # You can always obtain a callback manager for this by calling
        # `run_manager.get_child()` as shown below.
        response = await self.llm.agenerate_prompt(
            [prompt_value],
            callbacks=run_manager.get_child() if run_manager else None
        )

        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        if run_manager:
            await run_manager.on_text("Log something about this run")
        
        return {self.output_key: response.generations[0][0].text}

    @property
    def _chain_type(self) -> str:
        return "my_custom_chain"
    
    
def not_compilable(solidity_code_str):
    return "Compilation Failed" in compile_solidity(solidity_code_str)
        
    
from pydantic import BaseModel, Field
from solcx import install_solc, compile_source


def compile_solidity(solidity_code_str):
    """
    This function compiles Solidity files using a specific version of solc.
    :return: Dict, the compiled contracts.
    """
    solidity_code_str = solidity_code_str.strip("`")
    solc_version = "0.8.9"
    # Install the specified version of solc
    install_solc(solc_version)
    try:
        result = compile_source(solidity_code_str)
        print(result)
        return "Compilation Successful. It is a valid solidity code."
    except Exception as e:
        print(f"Couldn't compile {solidity_code_str}\n{str(e)}")
        return f"Compilation Failed. Either the code is invalid or current compiler({solc_version}) didnt match"


In [6]:
solidity_function = """
function transfer(address recipient, uint amount) external returns (bool) {
    balanceOf[msg.sender] -= amount;
    balanceOf[recipient] += amount;
    emit Transfer(msg.sender, recipient, amount);
    return true;
    }
"""

from langchain.callbacks.stdout import StdOutCallbackHandler
from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate


chain = MyCustomChain(
    prompt=PromptTemplate.from_template('write a test function to this solidity function {solidity_function}'),
    llm=ChatOpenAI()
)

chain.run({'solidity_function': solidity_function}, callbacks=[StdOutCallbackHandler()])




> Entering new MyCustomChain chain...
Couldn't compile 
function transfer(address recipient, uint amount) external returns (bool) {
    balanceOf[msg.sender] -= amount;
    balanceOf[recipient] += amount;
    emit Transfer(msg.sender, recipient, amount);
    return true;
    }

An error occurred during execution
> command: `/Users/uan/anaconda3/envs/agents/bin/solc --combined-json abi,asm,ast,bin,bin-runtime,compact-format,devdoc,function-debug,function-debug-runtime,generated-sources,generated-sources-runtime,hashes,interface,metadata,opcodes,srcmap,srcmap-runtime,storage-layout,userdoc -`
> return code: `1`
> stdout:

> stderr:
--> <stdin>

Error: Free functions cannot have visibility.
 --> <stdin>:2:1:
  |
2 | function transfer(address recipient, uint amount) external returns (bool) {
  | ^ (Relevant source part starts here and spans across multiple lines).

--> <stdin>

Error: Undeclared identifier.
 --> <stdin>:3:5:
  |
3 |     balanceOf[msg.sender] -= amount;
  |     ^^^^^^^^^


'not compilable'

In [10]:
# ops forgot adding pragma
# the reason why I don't use the latest 0.8.20 for this example: https://stackoverflow.com/questions/70505253/solcx-compile-source-is-throwing-error-error-occurred-during-execution
solidity_function = """// SPDX-License-Identifier: MIT
pragma solidity ^0.8.9;

contract Gas {
    uint public i = 0;

    // Using up all of the gas that you send causes your transaction to fail.
    // State changes are undone.
    // Gas spent are not refunded.
    function forever() public {
        // Here we run a loop until all of the gas are spent
        // and the transaction fails
        while (true) {
            i += 1;
        }
    }
}
"""
chain.run({'solidity_function': solidity_function}, callbacks=[StdOutCallbackHandler()])



> Entering new MyCustomChain chain...
{'<stdin>:Gas': {'abi': [{'inputs': [], 'name': 'forever', 'outputs': [], 'stateMutability': 'nonpayable', 'type': 'function'}, {'inputs': [], 'name': 'i', 'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}], 'stateMutability': 'view', 'type': 'function'}], 'asm': {'.code': [{'begin': 57, 'end': 432, 'name': 'PUSH', 'source': 0, 'value': '80'}, {'begin': 57, 'end': 432, 'name': 'PUSH', 'source': 0, 'value': '40'}, {'begin': 57, 'end': 432, 'name': 'MSTORE', 'source': 0}, {'begin': 92, 'end': 93, 'name': 'PUSH', 'source': 0, 'value': '0'}, {'begin': 76, 'end': 93, 'name': 'DUP1', 'source': 0}, {'begin': 76, 'end': 93, 'name': 'SSTORE', 'source': 0}, {'begin': 57, 'end': 432, 'name': 'CALLVALUE', 'source': 0}, {'begin': 57, 'end': 432, 'name': 'DUP1', 'source': 0}, {'begin': 57, 'end': 432, 'name': 'ISZERO', 'source': 0}, {'begin': 57, 'end': 432, 'name': 'PUSH [tag]', 'source': 0, 'value': '1'}, {'begin': 57, 'end': 432, 'name'

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 0176da6cf1c15a8bcd83dd90a77cf6f6 in your message.).


Log something about this run
> Finished chain.


'// SPDX-License-Identifier: MIT\npragma solidity ^0.8.9;\n\nimport "truffle/Assert.sol";\nimport "../contracts/Gas.sol";\n\ncontract TestGas {\n    Gas gasContract;\n\n    function beforeEach() public {\n        gasContract = new Gas();\n    }\n\n    function testForever() public {\n        bool success;\n\n        try gasContract.forever() {\n            success = true;\n        } catch {\n            success = false;\n        }\n\n        Assert.isFalse(success, "Transaction should fail and revert");\n        Assert.equal(gasContract.i(), 0, "State changes should be undone");\n    }\n}'